In [1]:

%load_ext bigdata

In [2]:
%hive_start

In [3]:

%timeout 300

In [4]:
!hdfs dfs -ls /tmp

Found 2 items
drwxrwx---   - root supergroup          0 2020-01-20 00:47 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-01-20 00:49 /tmp/hive


In [5]:
!hdfs dfs -mkdir /tmp/hivep3

In [16]:
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-01-20 00:47 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-01-20 00:49 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-01-20 00:52 /tmp/hivep2
drwxr-xr-x   - root supergroup          0 2020-01-20 00:52 /tmp/hivep3


In [10]:
!hdfs dfs -copyFromLocal data.tsv /tmp/hivep3/

In [22]:
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/hivep3

Found 1 items
-rw-r--r--   1 root supergroup        677 2020-01-20 00:32 /tmp/hivep3/data.tsv


In [7]:
%%hive
DROP TABLE IF EXISTS data;

CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)

ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';

DROP TABLE IF EXISTS data;
OK
Time taken: 9.606 seconds
CREATE TABLE data (letra       STRING,
                           fecha        STRING,
                           numero      INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 0.736 seconds


In [8]:
%%hive
SHOW CREATE TABLE data;

SHOW CREATE TABLE data;
OK
CREATE TABLE `data`(
  `letra` string, 
  `fecha` string, 
  `numero` int)
ROW FORMAT SERDE 
  'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe' 
WITH SERDEPROPERTIES ( 
  'field.delim'='\t', 
  'serialization.format'='\t') 
STORED AS INPUTFORMAT 
  'org.apache.hadoop.mapred.TextInputFormat' 
OUTPUTFORMAT 
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  'hdfs://0.0.0.0:9000/user/hive/warehouse/data'
TBLPROPERTIES (
  'transient_lastDdlTime'='1579481399')
Time taken: 0.292 seconds, Fetched: 17 row(s)


In [11]:
%%hive
LOAD DATA INPATH '/tmp/hivep3/data.tsv' OVERWRITE
INTO TABLE data;

LOAD DATA INPATH '/tmp/hivep3/data.tsv' OVERWRITE
INTO TABLE data;
Loading data to table default.data
OK
Time taken: 1.162 seconds


In [12]:
%%hive
SHOW TBLPROPERTIES data;

SHOW TBLPROPERTIES data;
OK
numFiles	1
numRows	0
rawDataSize	0
totalSize	677
transient_lastDdlTime	1579481467
Time taken: 0.031 seconds, Fetched: 5 row(s)


In [13]:
%%hive
SELECT * FROM data;

SELECT * FROM data;
OK
B	1999-08-28	14
E	1999-12-06	12
E	1993-07-21	17
C	1991-02-12	13
E	1995-04-25	16
A	1992-08-22	14
B	1999-06-11	12
E	1993-01-27	13
E	1999-09-10	11
E	1990-05-03	16
E	1994-02-14	5
A	1988-04-27	12
A	1990-10-06	10
E	1985-02-12	16
E	1998-09-14	16
B	1994-08-30	17
A	1997-12-15	13
B	1995-08-23	10
B	1998-11-22	13
B	1997-04-09	14
E	1993-12-27	18
E	1999-01-14	15
A	1992-09-19	18
B	1993-03-02	14
B	1999-10-21	13
A	1990-08-31	12
C	1994-01-25	6
E	1990-02-09	18
A	1990-09-26	14
A	1993-05-08	16
B	1995-09-06	14
E	1991-02-18	14
A	1993-01-11	14
A	1990-07-22	18
C	1994-09-09	15
C	1994-07-27	7
D	1990-10-10	15
A	1990-09-05	11
B	1991-10-01	15
A	1994-10-25	13
Time taken: 1.548 seconds, Fetched: 40 row(s)


In [25]:
%%hive
DROP TABLE IF EXISTS word_counts;
CREATE TABLE word_counts
AS 
SELECT numero 
    FROM data
GROUP BY
     numero
ORDER BY  NUMERO  LIMIT 5
       

;


INSERT OVERWRITE DIRECTORY '/tmp/hivep3'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM word_counts;

DROP TABLE IF EXISTS word_counts;
OK
Time taken: 0.871 seconds
CREATE TABLE word_counts
AS 
SELECT numero 
    FROM data
GROUP BY
     numero
ORDER BY  NUMERO  LIMIT 5
;
Query ID = root_20200120005950_106a0abb-ef28-4cb8-af82-578dd798978f
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579481217016_0006, Tracking URL = http://10f3f3d27e8d:8088/proxy/application_1579481217016_0006/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579481217016_0006
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-01-20 01:00:05,103 Stage-1 map = 0%,  reduce = 0%
2020-01-20 01:00:14,746 Stage-1 ma

In [26]:
## Ejecucion
!hive -S -e 'source hivep3.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: hivep3.hql is not a file.


In [18]:

## Se lista el contenido del archivo.
!hdfs dfs -ls /tmp/hivep3

Found 1 items
-rwxr-xr-x   1 root supergroup         15 2020-01-20 00:52 /tmp/hivep3/000000_0


In [27]:
!hdfs dfs -cat /tmp/hivep3/000000_0 

5
6
7
10
11


In [28]:
!hdfs dfs -copyToLocal /tmp/hivep3/000000_0 output;